# Нейронная сеть, которая будет рекомендовать соотношение матрица-наполнитель. 

In [66]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler



In [67]:
# Загрузка данных
df = pd.read_csv('VKR.csv')

In [68]:
X = df.drop(['Соотношение матрица-наполнитель'], axis=1)
y = df[['Соотношение матрица-наполнитель']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [69]:
# Создание объекта MinMaxScaler
scaler = MinMaxScaler()
# Масштабирование целевой переменной
y_train_scaled = scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler.transform(y_test.values.reshape(-1, 1))

In [70]:
# Создание модели нейронной сети
model = Sequential()

In [71]:
# Добавление слоев
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))

In [72]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 64)                832       
                                                                 
 dropout_21 (Dropout)        (None, 64)                0         
                                                                 
 dense_29 (Dense)            (None, 32)                2080      
                                                                 
 dropout_22 (Dropout)        (None, 32)                0         
                                                                 
 dense_30 (Dense)            (None, 16)                528       
                                                                 
 dropout_23 (Dropout)        (None, 16)                0         
                                                                 
 dense_31 (Dense)            (None, 1)                

In [73]:
# Компиляция модели
model.compile(loss='mse', optimizer='adam')

In [74]:
# Обучение модели
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
history = model.fit(X_train_scaled, y_train, epochs=100, validation_split=0.2, callbacks=[early_stop])

Epoch 1/100
18/18 [==============================] - 1s 23ms/step - loss: 0.0795 - val_loss: 0.0344
Epoch 2/100
18/18 [==============================] - 0s 6ms/step - loss: 0.0588 - val_loss: 0.0362
Epoch 3/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0527 - val_loss: 0.0396
Epoch 4/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0496 - val_loss: 0.0369
Epoch 5/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0459 - val_loss: 0.0358
Epoch 6/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0464 - val_loss: 0.0353
Epoch 7/100
18/18 [==============================] - 0s 8ms/step - loss: 0.0434 - val_loss: 0.0364
Epoch 8/100
18/18 [==============================] - 0s 7ms/step - loss: 0.0418 - val_loss: 0.0348
Epoch 9/100
18/18 [==============================] - 0s 8ms/step - loss: 0.0414 - val_loss: 0.0313
Epoch 10/100
18/18 [==============================] - 0s 8ms/step - loss: 0.0409 - val_loss: 0.0337
Epoch 11

In [75]:
# Оценка качества модели на тестовых данных
y_pred = model.predict(X_test_scaled)
mse = np.mean((y_pred - y_test)**2)
print(f"Средняя квадратичная ошибка на тестовых данных: {mse.item():.4f}")


10/10 [==============================] - 0s 3ms/step
Средняя квадратичная ошибка на тестовых данных: 0.0311


C:\Users\ASUS\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [77]:
# Обратное масштабирование предсказанных значений
y_pred_unscaled = scaler.inverse_transform(y_pred)

Средняя квадратичная ошибка (MSE) на тестовых данных составляет 0.0311, что означает, что наша модель хорошо справляется с прогнозированием соотношения матрица-наполнитель на основе имеющихся данных. Однако, для более точной оценки качества модели, может потребоваться использование других метрик и методов оценки.

In [78]:
model.save('my_model.h5')
